# Transaction Cost Analysis
## Comprehensive Analysis of Transaction Costs Impact on Portfolio Performance

This notebook demonstrates how to:
1. Analyze portfolio performance under different cost structures
2. Find optimal rebalancing frequencies
3. Compare strategies with and without transaction costs
4. Generate comprehensive reports and visualizations

In [ ]:
import sys

sys.path.append("..")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from code.transaction_cost_analysis import TransactionCostAnalyzer

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (14, 8)

print("Imports successful!")

## Step 1: Initialize Analyzer

In [ ]:
# Initialize analyzer
analyzer = TransactionCostAnalyzer(config_path="../config/config.yaml")

print("Cost Structures:", analyzer.cost_structures)
print("Rebalancing Frequencies:", analyzer.rebalance_frequencies)

## Step 2: Load Portfolio Data

Load portfolio values and weights history from your trained models.

In [ ]:
# Example: Load from saved results
# In practice, load from your evaluation results

# Dummy data for demonstration
n_periods = 252  # One year of trading days
initial_value = 1000000

# Simulate portfolio performance
returns = np.random.normal(0.0004, 0.01, n_periods)  # Daily returns
portfolio_values = [initial_value]
for r in returns:
    portfolio_values.append(portfolio_values[-1] * (1 + r))

# Simulate weight changes
n_assets = 25
portfolio_weights = []
for i in range(len(portfolio_values)):
    if i == 0:
        weights = np.ones(n_assets) / n_assets
    else:
        # Small random changes
        weights = weights + np.random.normal(0, 0.01, n_assets)
        weights = np.clip(weights, 0, 1)
        weights = weights / weights.sum()
    portfolio_weights.append(weights)

dates = pd.date_range("2023-01-01", periods=len(portfolio_values), freq="D")

print(f"Portfolio data loaded: {len(portfolio_values)} periods")
print(f"Initial value: ${initial_value:,.0f}")
print(f"Final value (no costs): ${portfolio_values[-1]:,.0f}")

## Step 3: Analyze Rebalancing Frequency Impact

In [ ]:
# Run comprehensive analysis
results_df = analyzer.analyze_rebalancing_frequency(
    strategy_name="PPO",
    portfolio_values_base=portfolio_values,
    portfolio_weights_history=portfolio_weights,
    dates=dates,
)

print("\nAnalysis Results:")
print(
    results_df[
        ["frequency_name", "cost_structure", "cost_impact_pct", "sharpe_with_cost"]
    ].head(10)
)

## Step 4: Visualize Results

In [ ]:
# Generate comprehensive visualization
fig = analyzer.plot_cost_impact(
    results_df, save_path="../results/figures/transaction_cost_impact.png"
)
plt.show()

## Step 5: Find Optimal Rebalancing Frequency

In [ ]:
# Find optimal frequency for each cost structure
optimal = results_df.loc[
    results_df.groupby("cost_structure")["sharpe_with_cost"].idxmax()
]

print("\nOptimal Rebalancing Frequencies:")
print("=" * 60)
for _, row in optimal.iterrows():
    print(f"\nCost Structure: {row['cost_structure']}")
    print(f"  Optimal Frequency: {row['frequency_name']}")
    print(f"  Sharpe Ratio: {row['sharpe_with_cost']:.3f}")
    print(f"  Cost Impact: {row['cost_impact_pct']:.2f}%")
    print(f"  Total Costs: ${row['total_transaction_costs']:,.0f}")

## Step 6: Generate Comprehensive Report

In [ ]:
# Generate detailed report
report = analyzer.generate_cost_report(
    results_df, output_path="../results/reports/transaction_cost_report.txt"
)

print(report)

## Key Insights

### Cost Impact by Structure:
- **Retail (0.5%)**: Significant drag on returns, especially with frequent rebalancing
- **Standard (0.1%)**: Moderate impact, manageable with appropriate frequency
- **Institutional (0.05%)**: Minimal impact, allows more frequent rebalancing

### Optimal Rebalancing:
- **Daily**: Only viable for institutional costs
- **Weekly**: Good balance for standard costs
- **Monthly**: Reduces costs but may miss opportunities

### Recommendations:
1. For retail investors: Rebalance monthly to quarterly
2. For institutions: Weekly rebalancing optimal
3. Consider tax implications in addition to transaction costs
4. Use limit orders to reduce slippage